# Kredibilitetsintervall

Alle fordelingene vi har brukt gir oss tilgang til en "interval"-metode. Denne finner det tosidige kredibilitetsintervallet med eneste parameter alfa. Du oppgir først parameterne til sannsynlighetsfordelingen (normalfordeling, student's T osv), som tidligere, og kaller deretter interval-metoden.

### Normalfordelingen
Vi lager en normalfordeling med $\mu=0.5$ og $\sigma=5.4$, og finner det 89% tosidige kredibilitetsintervallet.

In [1]:
from scipy.stats import norm

norm(0.5, 5.4).interval(0.89)

(-8.1302429555832152, 9.1302429555832205)

### Student's T

Merk at rekkefølgen på parameterne i Sipy.stats ikke er lik den som brukes i læreboka. Utover dette, fungerer utregning av tosidig intervall på samme måte som for normalfordelingen. Her bruker vi $\nu=10$, $\mu=22.6$ og $\sigma=3.5$

In [2]:
from scipy.stats import t

t(10, 22.6, 3.5).interval(0.998)

(8.0970482708368205, 37.102951729163181)

### Eksponentialfordelingen

Over brukte vi interval-metoden. Denne gir som nevnt tosidig kredibilitetsintervall. Hvis vi vil ha et ensidig intervall, bruker vi invers, eller PPF (se kapittel om kontinuerlige sannsynlighetsfordelinger). Vi skal her finne venstre ensidige kredibilitetsintervall.

Merk at også denne avviker fra lærebok og formelsamling ved at vi må oppgi $\mu$ og ikke $\lambda$. Dette gjør vi ved å ta $\frac{1}{\lambda}$. Så hvis vi har lambda = 2.34, kan vi gjøre følgende (95% kredibilitetsintervall):

In [3]:
from scipy.stats import expon

(0, expon(scale=1/2.34).ppf(0.95))

(0, 1.280227467330765)

# Hypotesetest

### Eksempel 1

Vi får oppgitt at $X \tilde {} \Phi_{(0.2, 4.7)}$. Den konservative hypotesen er at $X \ge 5$. Finn resultatet av testen med signifikans $\alpha=0.075$.

In [4]:
X = norm(0.2, 4.7)
alpha = 0.075

5 < X.ppf(1-alpha)

True

### Eksempel 2

Vi har $Y_1 \tilde {} St_{(5.2, 1.4, 11)}$ og $Y_2 \tilde {} St_{(-3.78, 3, 12)}$. Hypotesen er at $Y_1=Y_2$ med signifikans $\alpha=0.01$.

Her må vi finne differansen mellom to fordelinger (her Student's T-fordelinger), og vi må ha en hjelpefunksjon (diff_t). $\mu$ og $\sigma$ er greie, men vi trenger også Satterthwaites formel, som regner ut df ($\nu$). Deretter kan vi bruke diff_t-funksjonen med parameterne for de to fordelingene vi skal finne differansen av, og få en t-fordeling som returverdi.

Når vi har denne, bruker vi interval-metoden som vi har sett før, og sjekker om 0 ligger inne i intervallet.

In [5]:
from math import floor

# Basert på 8.2 i formelsamling (side 14 i 2016-utgaven)
def diff_t(df1, mu1, sigma1, df2, mu2, sigma2):
    mu = mu1 - mu2
    sigma = (sigma1**2 + sigma2**2)**0.5
    
    # Regne ut ny / degrees of freedom med Satterthwaites formel
    df = floor(
        (sigma1**2/(df1+1) + sigma2**2/(df2+1))**2 /
        ((sigma1**2/(df1+1))**2/df1 + (sigma2**2/(df2+1))**2/df2)
    )
    return t(df, mu, sigma)

# Ny fordeling, differansen mellom to t-fordelinger (parameterrekkefølge: ny, my, sigma for begge fordelinger)
Z = diff_t(11, 5.2, 1.4, 12, -3.78, 3)

# Finn intervall
alpha = 0.01
fra, til = Z.interval(1-alpha)

# Sjekk om 0 ligger innenfor intervallet
H_0 = fra < 0 < til

# Svar på tekstform
if H_0:
    print('Nullhypotesen stemmer, og skal ikke forkastes.')
else:
    print('Nullhypotesen forkastes.')

Nullhypotesen stemmer, og skal ikke forkastes.


### Eksempel 3

Vi får oppgitt $X \tilde {} Exp_{(2.34)}$, og vår konservative hypotese er at $X \ge 1$. Avgjør hypotesetesten med signifikans $\alpha=0.05$.

Her må vi som vanlig oppgi $\mu$ og ikke $\lambda$, så vi tar $\frac{1}{\lambda}$. Vi bruker X sin ppf-metode (som tidligere, ved ensidig intervall). Vi er ute etter venstre side. H_0-variabelen er en sammensatt sammenligning og sjekker først om 1 er større enn 0, og om det er mindre enn resultater fra PPF-funksjonen (øvre grense i intervallet). Resultatet blir en boolean, altså sant/usant, og vi bruker en if-test for å få et svar på tekstform. Hvis 1 er innenfor intervallet, får vi beskjed om at nullhypotesen stemmer.

In [6]:
X = expon(scale=1/2.34) # Vår fordeling / stokastiske variabel
alpha = 0.05 # Signifikans

# Sjekk om 1 er innenfor intervallet
H_0 = 0 < 1 < X.ppf(1-alpha) # 1-alpha for å få venstre intervall

# Svar på tekstform
if H_0:
    print('Nullhypotesen stemmer, og skal ikke forkastes.')
else:
    print('Nullhypotesen forkastes.')

Nullhypotesen stemmer, og skal ikke forkastes.
